# Knapsack problem - Genetic algorithm demo

### Knapsack problem solution using a simple genetic algorithm library

For the demo we are going to play the role of a capitain of a cargo ship, the mighty S.S.Doodle.

<img src="img/ssdoodle.png">

(sorry for the over complicated boat schematic)

In our demo, as capitain, we are in charge of balance of cargo. We have to choose which containers we ship depending on the weight and the value of their contents.
The trick is that we have to keep the weigth below the maximum while maintaining the total value of the cargo as high as possible to make profit of the travel.
This kind of problem is a combinatorial optimization problem called the Knapsack problem ([Wiki](https://en.wikipedia.org/wiki/Knapsack_problem)).

We are going to find a solution to this problem using a very simple genetic algorithm library.

Let's start defining an _elements_ dictionary with all the posible container types and a *MAX_WEIGHT* variable with the weight limit of our ship

In [1]:
elements = {
    "A": (3, 2),  # (price, weight)
    "B": (10, 7),
    "C": (23, 11),
    "Z": (0, 0)  # we can leave a position empty if we want
}
MAX_WEIGHT = 315
MAX_CONTAINERS = 30


The *totalvalues* function will help us with the evaluation of the containers

In [2]:
def totalvalues(x):
    totalw, totalp = 0, 0
    for container in x:
        values = elements[container]
        totalp += values[0] * values[1]  # price per weight
        totalw += values[1]
    return totalw, totalp

Now we need a fitness function to tell how good a given cargo selection is.

In [3]:
def fitness(x):
    totalw, totalp = totalvalues(x)  # sum of total weight and value of the cargo configuration
    if totalw <= MAX_WEIGHT:  # if under max weight return it's value
        return totalp * totalw  # multiply so the fitness scales linearly
    else:  # return it's owerweight as negative value
        ow = totalw - MAX_WEIGHT
        return -ow

We are ready to create a reactor that finds a solution for the cargo

In [4]:
from reactor import Reactor

solver = Reactor(
    6,                      # number of configurations cycling inside the reactor
    MAX_CONTAINERS,         # number of elements in the solution
    list(elements.keys()),  # list with the representation of the containers
    fitness,                # our fitness function
    0.4,                    # probability of mutation inside the reactor
    engine="thread"         # we use threads because jupyter doesn't allow multiprocessing
)

solver.addCores(3)  # set reactor to compute 4 solutions in paralell
solver.run(10000)  # cycles to run
res = solver.get_results()  # wait for the results

print(res)

[['Z', 'C', 'C', 'A', 'C', 'C', 'C', 'C', 'C', 'A', 'C', 'C', 'C', 'C', 'B', 'C', 'C', 'A', 'C', 'Z', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C'], ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'A', 'C', 'C', 'C', 'A', 'Z', 'C', 'C', 'C', 'B', 'C', 'C', 'C', 'Z', 'C', 'C', 'C', 'C', 'B', 'C', 'C'], ['C', 'C', 'B', 'C', 'C', 'C', 'B', 'Z', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'B', 'C', 'C', 'C', 'C', 'A', 'C', 'B', 'C', 'B', 'C', 'C'], ['C', 'C', 'B', 'C', 'B', 'C', 'B', 'C', 'C', 'C', 'C', 'C', 'B', 'C', 'Z', 'C', 'C', 'C', 'C', 'B', 'C', 'A', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']]


The results are lists with letters representing the containers, let's analise and show them on a clearer way.

In [5]:
for i in range(len(res)):
        solution = res[i]
        print("Solution {}:".format(i))
        print("\tFitness: {}".format(fitness(solution)))
        print("\tValues (W, P): {}".format(totalvalues(solution)))
        sortedRes = sorted(solution, key=lambda x: elements[x][1])  # sort by weight
        for j in range(0, len(sortedRes), 5):
            print("\t\t##### ##### ##### ##### #####")
            print("\t\t# {} # # {} # # {} # # {} # # {} #".format(sortedRes[j], sortedRes[j+1], sortedRes[j+2], sortedRes[j+3], sortedRes[j+4]))
            print("\t\t##### ##### ##### ##### #####")

Solution 0:
	Fitness: 1706320
	Values (W, P): (277, 6160)
		##### ##### ##### ##### #####
		# Z # # Z # # A # # A # # A #
		##### ##### ##### ##### #####
		##### ##### ##### ##### #####
		# B # # C # # C # # C # # C #
		##### ##### ##### ##### #####
		##### ##### ##### ##### #####
		# C # # C # # C # # C # # C #
		##### ##### ##### ##### #####
		##### ##### ##### ##### #####
		# C # # C # # C # # C # # C #
		##### ##### ##### ##### #####
		##### ##### ##### ##### #####
		# C # # C # # C # # C # # C #
		##### ##### ##### ##### #####
		##### ##### ##### ##### #####
		# C # # C # # C # # C # # C #
		##### ##### ##### ##### #####
Solution 1:
	Fitness: 1755168
	Values (W, P): (282, 6224)
		##### ##### ##### ##### #####
		# Z # # Z # # A # # A # # B #
		##### ##### ##### ##### #####
		##### ##### ##### ##### #####
		# B # # C # # C # # C # # C #
		##### ##### ##### ##### #####
		##### ##### ##### ##### #####
		# C # # C # # C # # C # # C #
		##### ##### ##### ##### #####
		##### ##### ##### 

As we can see the reactor outputs 4 solutions (one per core) to the problem. All are under the maximum weight limit, but some are better than the others depending on their total value. The better the cargo configuration, the higher the fitness value.